In [1]:
!pip install --no-deps unsloth vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.1/294.1 MB 7.4 MB/s eta 0:00:00


In [2]:
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-a_9xgdlz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-a_9xgdlz
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936469 sha256=b66b61db737b0cdb8089c8e804453ea69f7bb7b6ccdea749cd5e57ac0642c0f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-3rbkslp8/wheels/53/15/d5/d63b866c641d8863f9cd29a4cc7a5efc38476c3aae8247c195
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transf

In [3]:
!pip install --no-deps unsloth vllm
# [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
# Skip restarting message in Colab
import sys, re, requests; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
!pip install git+https://github.com/huggingface/trl@main
!pip install --no-deps bitsandbytes "accelerate>=0.28" xformers==0.0.29.post3 peft triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

# vLLM requirements - vLLM breaks Colab due to reinstalling numpy
f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
with open("vllm_requirements.txt", "wb") as file:
    file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
!pip install -r vllm_requirements.txt

  Cloning https://github.com/huggingface/trl (to revision main) to /tmp/pip-req-build-79p1vjpf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl /tmp/pip-req-build-79p1vjpf
  Resolved https://github.com/huggingface/trl to commit a9b27f82d67c366f421638941a05f86363b255b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.5 MB/s eta

In [4]:
!pip install --upgrade together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 10.0 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import re
from trl import GRPOConfig, GRPOTrainer
import time
# from huggingface_hub import login
import wandb
from datetime import datetime
from pathlib import Path
from transformers import logging as hf_logging
import json
from dataclasses import asdict, is_dataclass
from unsloth import FastModel
from unsloth import FastLanguageModel, is_bfloat16_supported
from together import Together
# from together.errors import RateLimitError
import httpx, requests

# # Huggingface Login
# login(token="")

# Wandb Login # 
wandb.login(key='YOUR_WANDB_KEY')
run = wandb.init(
    # Set the project where this run will be logged
    project="rl_course_project_llm_judge",
    dir = "/content/drive/MyDrive/rl_project/baseline_checkpooints_gemma_llm/wandb_logs"
)

# Huggingface cache dir
hf_cache_dir = "/content/drive/MyDrive/rl_project/hf_cache"
unsloth_cache_dir = "/content/drive/MyDrive/rl_project/unsloth_cache"

# Together API
together_client = Together(api_key="YOUR_TOGETHER_KEY")

INFO 04-25 01:33:28 [__init__.py:239] Automatically detected platform cuda.


<ipython-input-3-d324a39ed6d3>:15: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ax2119 (myprojs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
#####################################################
# LOADING THE MODEL
max_seq_length = 1024
lora_rank = 32
model_id = "unsloth/gemma-3-4b-it"

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length, # We can choose longer for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # A bit more accurate, uses 2x memory
    full_finetuning = False,
    cache_dir = unsloth_cache_dir,
)

model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always

    r = lora_rank,           # Larger = higher accuracy, but might overfit
    lora_alpha = lora_rank,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

print(f"\n{model_id} Trainabale Parameters --> ",model.print_trainable_parameters())

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Unsloth: Making `model.base_model.model.language_model.model` require gradients
trainable params: 59,604,992 || all params: 4,359,684,464 || trainable%: 1.3672

unsloth/gemma-3-4b-it Trainabale Parameters -->  None


In [5]:
#####################################################
# LOAD DATASET
dataset = load_dataset("openai/gsm8k", "main", split = "train")

#####################################################
# STRUCTURING THE DATASET
print("\nStructure of the dataset: ",dataset)

# One sample from the dataset
print("\nOne sample (first row) from the dataset: ",dataset[0])

# We notice all answers like about have a ####, so we extract it:
def extract_hash_answer(text):
    if "####" not in text: return None
    return text.split("####")[1].strip()

# We now create a system prompt which can be customized. We add 4 extra symbols for working out or thinking / reasoning sections and a final answer:
reasoning_start = "<think>"
reasoning_end   = "</think>"
solution_start = "<answer>"
solution_end = "</answer>"

system_prompt = \
f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant
first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning
process and answer are enclosed within {reasoning_start} {reasoning_end} and {solution_start} {solution_end} tags, respectively, i.e.,
{reasoning_start} reasoning process here {reasoning_end} {solution_start} answer here {solution_end}.
There should be only one instance of {reasoning_start} {reasoning_end} and {solution_start} {solution_end} tags, and not more in the correct output."""

# Let's map the dataset! and see the first row:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["question"]},
    ],
    "answer": extract_hash_answer(x["answer"]),
})
print("\nAfter mapping the first row",dataset[0])


Structure of the dataset:  Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

One sample (first row) from the dataset:  {'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

After mapping the first row {'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': '72', 'prompt': [{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant\nfirst thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning\nprocess and answer are enclosed within <think> </think> and <answer> </answer> 

In [6]:
#####################################################
# SETTING THE REWARD FUNCTIONS

# We create a regex format to match the reasoning sections and answers:
match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

# Format Reward
# We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 1.0
        scores.append(score)
    return scores

In [7]:
def _wait_time(resp, attempt, default=5.0):
    """Seconds to sleep = header + small back-off."""
    try:
        reset = float(resp.headers.get("x-ratelimit-reset", default))
    except (TypeError, ValueError):
        reset = default
    return reset + 0.5 * attempt          # exponential-ish buffer


def safe_chat(model: str,
              messages: list,
              max_retries: int = 3,
              client=None):
    """
    Call Together chat-completion, retrying on HTTP 429.

    Returns the response object, or the string 'FAILED' on repeated 429s.
    """
    client = client or together_client

    for attempt in range(1, max_retries + 1):
        try:
            return client.chat.completions.create(model=model, messages=messages, temperature=0.1)

        # Newer Together SDK uses httpx internally
        except httpx.HTTPStatusError as e:
            if e.response.status_code != 429:
                return "FAILED"                     # some other HTTP error
            wait = _wait_time(e.response, attempt)
            print(f"[429] sleeping {wait:.1f}s (attempt {attempt}/{max_retries})")
            time.sleep(wait)

        # Very old SDK versions still raise requests.HTTPError
        except requests.HTTPError as e:
            if e.response is None or e.response.status_code != 429:
                return "FAILED"
            wait = _wait_time(e.response, attempt)
            print(f"[429] sleeping {wait:.1f}s (attempt {attempt}/{max_retries})")
            time.sleep(wait)

    # all retries exhausted
    return "FAILED"

# match_numbers = re.compile(
#     rf"{solution_start}.*?([-+]?[\d\.eE]+)", flags=re.MULTILINE | re.DOTALL
# )

number_pat = (
    r"[-+]?"                      # optional sign
    r"(?:\d+\.\d*|\.\d+|\d+)"     # int or float (requires a digit somewhere)
    r"(?:[eE][-+]?\d+)?"          # optional exponent
)

match_numbers = re.compile(
    rf"{solution_start}.*?({number_pat})",
    flags=re.MULTILINE | re.DOTALL
)

score_tag = re.compile(r"<score>\s*([012])\s*</score>", re.I)

clean_text = lambda s: s.replace(",", "") # replace comma with blank

def _parse_score(txt: str) -> int | None:
    m = score_tag.search(txt)
    return int(m.group(1)) if m else None


def check_answer(prompts, completions, answer, **kwargs):
    judge_model      = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free"
    max_judge_tries  = 3
    question         = prompts[0][-1]["content"]
    responses        = [c[0]["content"] for c in completions]
    scores           = []

    print(f"question: {question},\nresponses: {responses},\nanswer: {answer}")

    system_prompt = """
      You are an impartial grader of mathematical reasoning.
      For each submission you will receive exactly two items:

      1. Question
      2. Student reasoning that claims to reach the correct final answer
        (The numeric answer itself has already been verified.)

      Your job: read the student’s reasoning, decide how sound it is, and assign a single integer score using the rubric below.

      Rubric
      0 – Reasoning is mostly wrong, off-topic, or unverifiable.
      1 – Reasoning contains some valid steps but also gaps, unjustified leaps, or errors.
      2 – Reasoning is fully correct, logically complete, and properly justified.

      Style penalty
      If the writing is flashy/ornate yet mathematically incoherent, subtract **1 point** after applying the rubric (minimum score = 0).

      Note: Do not penalize the students reasoning if it first went in the wrong direction initially and then corrected itself, that is actually a good sign.

      Output format (strict)
      Reply with nothing except <score>n</score> where n can be only from the set {0, 1, 2}.
      Do not output anything else.
      """

    for student_resp, true_ans in zip(responses, answer):
        # ---- 1) numeric check ----
        cleaned_resp = clean_text(student_resp)
        m = match_numbers.search(cleaned_resp)
        print(f"\m:{m}")
        if not m:
            scores.append(0.0)
            continue

        print_statemet = (
            "#############################"
          f"\n### Question:\n{question}\n\n"
          f"### Correct Answer:\n{true_ans}\n\n"
          f"### Student reasoning and answer:\n{cleaned_resp.strip()}\n\n"
          )

        print(print_statemet)

        try:
            guess_val = float(m.group(1).strip())
            true_val  = float(true_ans.strip().replace(",", ""))
            print(f"guess_val: {guess_val}, true_val: {true_val}")
        except ValueError:
            scores.append(0.0)
            continue

        if guess_val != true_val:
            scores.append(0.0)
            continue  # wrong answer → no need to call judge

        # ---- 2) call judge for reasoning score ----
        attempt, score = 0, None
        while attempt < max_judge_tries and score is None:
            attempt += 1
            judge_prompt = (
                f"{system_prompt}\n\n"
                f"### Question:\n{question}\n\n"
                f"### Correct Answer:\n{true_ans}\n\n"
                f"### Student reasoning and answer:\n{cleaned_resp.strip()}\n\n"
                "### Grader verdict:"
            )
            llm_reply = safe_chat(judge_model, [{"role": "user", "content": judge_prompt}])
            if llm_reply == "FAILED":
                print(f"[Judge attempt {attempt}] 429 retrying…")
                continue
            else:
              llm_reply = llm_reply.choices[0].message.content
            print("\nllm_reply: ",llm_reply)
            score = _parse_score(llm_reply)
            if score is None:
                print(f"[Judge attempt {attempt}] bad format => retrying…")

        scores.append(float(score if score is not None else 0))

    return scores

In [ ]:
hub_repo   = "aarjun0112/llm_judge_gemma_simple"           # create it once on the HF website
hf_token   = "YOUR_HF_KEY"   # or paste the token string here

run_stamp  = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = Path("/content") / hub_repo       # ⭐ clone repo locally as the output dir
!git lfs install --skip-smudge                 # avoids pulling large files on clone
!git clone https://huggingface.co/{hub_repo} {output_dir}

Git LFS initialized.
Cloning into '/content/aarjun0112/llm_judge_gemma_simple'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 3), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (28/28), 47.41 KiB | 2.37 MiB/s, done.


In [9]:
output_dir

PosixPath('/content/aarjun0112/llm_judge_gemma_simple')

In [10]:
#####################################################
# SETTING THE GRPO TRAINING HYPER-PARAMETERS
max_prompt_length = 384
max_completion_length = max_seq_length - max_prompt_length # 768 if max_seq_length - 1024

training_args = GRPOConfig(
    output_dir=str(output_dir),
    run_name = run_stamp,

    learning_rate = 5e-6,
    # adam_beta1 = 0.9,
    # adam_beta2 = 0.99,
    # weight_decay = 0.1,
    # warmup_ratio = 0.1,
    warmup_steps = 15,
    # lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    max_grad_norm = 0.2,

    push_to_hub     = True,                    # ← magic switch to save in hf hub
    hub_token       = hf_token,
    hub_model_id    = hub_repo,

    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),

    # loss_type = "dr_grpo",
    mask_truncated_completions = True,
    # scale_rewards=False,
    beta= 0.001,
    # num_iterations=2,

    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 6, # Increase for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,

    num_train_epochs = 5, # Set to 1 for a full training run

    report_to="wandb",
    logging_strategy="steps",         # more explicit
    logging_steps=10,
    logging_first_step=True,          # log step‑0 metrics
    log_level="info",                 # HF logger verbosity
    disable_tqdm=False,               # keep the progress bar

    save_strategy="steps",
    save_steps=20,
    save_total_limit=15,
)

# writes training_args.json
cfg_path = Path("/content/training_args.json")

# Works for both dataclass- and plain‑object configs
cfg_dict = asdict(training_args) if is_dataclass(training_args) else vars(training_args)

with cfg_path.open("w") as f:
    json.dump(cfg_dict, f, indent=2)

#####################################################
# TRAINING THE MODEL
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        # match_format_approximately,
        check_answer,
        # check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,
)

hf_logging.set_verbosity_info()                 # shows every call & warning

Using auto half precision backend


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 5 | Total steps = 37,365
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 6
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 6 x 1) = 6
 "-____-"     Trainable parameters = 59,604,992/4,359,684,464 (1.37% trained)
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


question: Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers?,
responses: ["Here’s the solution breakdown:\n\n<think>Ahmed needs to achieve a minimum score to surpass Emily's grade. Emily's score stands at 92, and Ahmed's current score is 91. Considering the final assignment holds the same weight as all others, and Emily achieved 90 on the final, Ahmed must achieve a score greater than 92 (to be able to beat Emily’s grade) with whole numbers. </think> <answer>Ahmed needs a minimum score of 93 to beat Emily's grade. </answer>", "Here's the solution breakdown:\n\n<think>Ahmed needs to figure out what grade he needs to beat Emily’s final grade of 90. Since Emily has a final 

Step,Training Loss
1,0.000000
10,0.000000
20,-0.169500


question: The gauge on a water tank shows that the tank is 1/3 full of water. To fill the tank, 16 gallons of water are added. How many gallons of water does the tank hold when full?,
responses: ["Here's the solution, incorporating the requested elements:\n\n<think>The tank is 1/3 full, and 16 gallons are added to fill it.  Therefore, 16 gallons represent 2/3s of the tank'ss capacity.  Let 'x' be the full capacity.  So, (2/3)x = 16.  Multiplying by 3/2, we get x = 16 * (3/2) = 24 gallons. </think> <answer>The tank holds 24 gallons when full. </answer>", 'Here’s the solution, incorporating the requested elements:\n\n<think>The tank is 1/3 full, and adding 16 gallons fills it. So, the tank holds 3 times 16 gallons, which is 48 gallons. </think> <answer>The tank holds 48 gallons when full.</answer>', "Here's an solution incorporating the constraints you've set up:\n\n<think>The problem states that 1/3 of the tank's current capacity holds 16 gallons of water. To find the tank’s full capaci

Saving model checkpoint to /content/aarjun0112/llm_judge_gemma_simple/checkpoint-20
Image processor saved in /content/aarjun0112/llm_judge_gemma_simple/checkpoint-20/preprocessor_config.json
chat template saved in /content/aarjun0112/llm_judge_gemma_simple/checkpoint-20/chat_template.json
processor saved in /content/aarjun0112/llm_judge_gemma_simple/checkpoint-20/processor_config.json
Image processor saved in /content/aarjun0112/llm_judge_gemma_simple/preprocessor_config.json
chat template saved in /content/aarjun0112/llm_judge_gemma_simple/chat_template.json
processor saved in /content/aarjun0112/llm_judge_gemma_simple/processor_config.json


question: Sebastian plays drums for percussion in an orchestra. He sits next to the brass section where four people play trombone, two play trumpet, and one plays a French horn. In front of them are the strings section with three violinists, a cellist, and a contrabassist. Next to the strings are a trio of clarinets and a quartet of flute players in the woodwinds section. The maestro conducts them all. How many people are in the orchestra?,
responses: ['Here’s an analysis of the provided information, followed by the solution, enclosed with the requested tags:\n\n<think>reasoning process here</think> <answer>answer here</answer>\n\n<think>The problem asks for the total number of people in the orchestra. We know the number of people in each section: Drum section(1 Sebastian), Brass section(4 trombone, 2 trumpet, 1 French horn), Strings section(3 violinists, 1 cellist, 1 contrabass). Woodwind section(3 clarinets, 1 quartet of flute players) .Total = 1+4+3+3+1=12</think> <answer>12</answer

In [ ]:
!mount | grep drive